In [1]:
import pickle
import pandas as pd
import numpy as np
from datetime import datetime
from collections import defaultdict

# Чтение данных

In [2]:
%%time 
players = pd.DataFrame(pickle.load(open('players.pkl', 'rb')).values())
cups = pd.DataFrame(pickle.load(open('tournaments.pkl', 'rb')).values())
cups['year'] = cups.dateStart.apply(lambda x: int(x[:4]))
cups = cups[(cups.year == 2019) | (cups.year == 2020)]
cups_present = {x for x in cups.id}
results = {}
for cup_id, values in pickle.load(open('results.pkl', 'rb')).items():
    if cup_id not in cups_present:
        continue
    players_results = {}
    teams_results = {}
    number_of_teams = len(values)
    questions_number = 0
    for team in values:
        if 'mask' not in team or not team['mask']:
            continue
        if 'position' not in team or not team['position']:
            continue
        position = team['position']
        team_id = team['team']['id']
        questions_number = questions_number or len(team['mask'])
        players_results.update({
            player['player']['id']: {
                'answer': team['mask'],
                'position': position / number_of_teams,
                'team': team_id
            } for player in team['teamMembers']})
        teams_results[team_id] = position
    if players_results:
        results[cup_id] = dict(
            players_results=players_results, 
            teams_results=teams_results,
            questions_number=questions_number
        )
cups_present = cups_present & set(results.keys())
cups = cups[cups.id.isin( cups_present)].reset_index(drop=True)
cups.dateStart = cups.dateStart.apply(datetime.fromisoformat)
cups.dateEnd = cups.dateEnd.apply(datetime.fromisoformat)
# results = pd.DataFrame(pickle.load(open('results.pkl', 'rb')).values())

CPU times: user 13.8 s, sys: 3.08 s, total: 16.9 s
Wall time: 18.5 s


# Baseline

В качестве бейзлайна обучим логистическую регрессию, которая должна будет предсказывать для данного игрока и данного вопроса вероятнсоть его правильного ответа на него. 
Для этого сформируем вектор признаков вопроса и вектор признаков, характеризующих игрока.

Так как единственная информация, доступная нам про вопрос, это характеристика турнира, то будем описывать вопрос признаками, соответствующими турниру:
* количство участников
* тип турнира
* длительность турнира

**Замечание:** можно было бы конечно использовать данные об ответе на вопрос (единственная информация, которая нам доступна по-вопросно из данных. Например, отношение правильных ответов к общему числу ответов на вопрос. Однако, с точки зрения модели такое делать некорректно: возникает утечка данных (подглядывание в ответы). Например, если бы доля правильных ответов равнялась 0, то это подсказало бы модели, что правильный ответ 0.

Вектор же игрока будем описывать его текущими достижениями:
* количество турниров, в которых принимал участие (деленное на общее количество турниров)
* количество турниров, в которых принимал участие, в лог шкале
* количество вопросов, на которые пытался дать ответ (деленное на общее количество вопросов)
* количество вопросов, на которые пытался дать ответ, в лог шкале
* среднее число правильных ответов
* среднее место, которое занимал участник в турнире (нормированное)
* дисперсия места в турнире
* максимальная позиция в турнире (нормированная)
* минимальная позиция в турнире (нормированная)

Выборка будет формироваться следующим образом: для каждого турнира, для каждого участника, и для каждого вопроса мы добавляем вектор фичей и лейбл. После турнира пересчитывааем вектор признаков участника: в след турнире у него обновится вектор результатов. Турниры будем перебирать в порядке возрастания.

Для проверки результатов - зафиксируем вектора участников по состоянию на конец 2019 года. Каждый турнир 2020 года будем рассматривать независимо.

### Подготовка данных

In [3]:
cups['competitors'] = cups.id.apply(lambda x: len(results[x]))
cups['competitors'] /= cups.competitors.max()
cups['cup_type'] = cups.type.apply(lambda x: x['id'])
cups['duration'] = (cups.dateEnd - cups.dateStart).apply(lambda x: x.total_seconds() / 3600)
cups['duration'] /= cups.duration.max()

cups_features = cups[['id', 'competitors', 'cup_type', 'duration']].copy().set_index('id')
cups_features = pd.concat([cups_features, pd.get_dummies(cups_features.cup_type, prefix='type')], axis=1)
cups_features.drop(columns='cup_type', inplace=True)

def get_cup_features(cup_id):
    global cups_features
    return cups_features.loc[cup_id, :].values

In [4]:
cups_features.head()

,competitors,duration,type_2,type_3,type_5,type_6,type_8
id,,,,,,,
4772,1.0,0.011545,0,1,0,0,0
4957,1.0,0.020084,0,1,0,0,0
4973,1.0,0.011535,0,1,0,0,0
4974,1.0,0.011535,0,1,0,0,0
4975,1.0,0.011535,0,1,0,0,0


In [5]:
players.head()

,id,name,patronymic,surname
0,1,Алексей,None,Абабилов
1,10,Игорь,,Абалов
2,11,Наталья,Юрьевна,Абалымова
3,12,Артур,Евгеньевич,Абальян
4,13,Эрик,Евгеньевич,Абальян


In [6]:
class PlayerFeatures(object):
    def __init__(self):
        self.cups_n = 0
        self.questions_n = 0
        self.correct_n = 0
        self.sum_position = 0.
        self.sum_square_position = 0.
        self.min_position = 1.
        self.max_position = 0.
        self.cups = defaultdict(int)
        
    def __call__(self, total_cups, total_questions):
        '''
        количество турниров, в которых принимал участие (деленное на общее количество турниров)
        количество турниров, в которых принимал участие, в лог шкале
        количество вопросов, на которые пытался дать ответ, (деленное на общее количество вопросов)
        количество вопросов, на которые пытался дать ответ, в лог шкале
        среднее число правильных ответов
        среднее место, которое занимал участник в турнире (нормированное)
        дисперсия места в турнире
        максимальная позиция в турнире (нормированная)
        минимальная позиция в турнире (нормированная)
        '''
        features = np.zeros(9)
        if self.cups_n == 0:
            return features
        average_position = self.sum_position / self.cups_n
        std = np.power((self.sum_square_position - self.cups_n * np.power(average_position, 2))/self.cups_n, 0.5)
        
        features[0] = self.cups_n / total_cups
        features[1] = np.log(self.cups_n)
        features[2] = self.questions_n / total_questions
        features[3] = np.log(self.questions_n)
        features[4] = self.correct_n / self.questions_n
        features[5] = average_position
        features[6] = std
        features[7] = self.max_position
        features[8] = self.min_position

        return features
    
    def update(self, cup_id, cup_position, answers):
        self.cups[cup_id] += 1
        
        self.cups_n += 1
        self.questions_n += len(answers)
        self.correct_n += answers.count('1')
        self.sum_position += cup_position
        self.sum_square_position += np.power(cup_position, 2)
        self.min_position = min(self.min_position, cup_position)
        self.max_position = max(self.max_position, cup_position)

PLAYER_FEATURES = defaultdict(PlayerFeatures)

def reset_player_features():
    global PLAYER_FEATURES
    PLAYER_FEATURES = defaultdict(PlayerFeatures)

def update_features(player_id, cup_id, cup_position, answers):
    PLAYER_FEATURES[player_id].update(cup_id, cup_position, answers)
    
def get_player_features(player_id, total_cups, total_questions):
    return PLAYER_FEATURES[player_id](total_cups, total_questions)
    

In [7]:
cups.head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty,year,competitors,cup_type,duration
0,4772,Синхрон северных стран. Зимний выпуск,2019-01-05 19:00:00+03:00,2019-01-09 19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2019-01-09T23:59:59...,"{'1': 12, '2': 12, '3': 12}",2019,1.0,3,0.011545
1,4957,Синхрон Биркиркары,2020-02-21 00:00:00+03:00,2020-02-27 23:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 2421, 'name': 'Ася', 'patronymic': 'Се...",{'dateRequestsAllowedTo': '2020-02-27T18:00:00...,"{'1': 13, '2': 13, '3': 13}",2020,1.0,3,0.020084
2,4973,Балтийский Берег. 3 игра,2019-01-25 19:05:00+03:00,2019-01-29 19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-01-28T23:59:59...,"{'1': 12, '2': 12, '3': 12}",2019,1.0,3,0.011535
3,4974,Балтийский Берег. 4 игра,2019-03-01 19:05:00+03:00,2019-03-05 19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-03-04T23:59:59...,"{'1': 12, '2': 12, '3': 12}",2019,1.0,3,0.011535
4,4975,Балтийский Берег. 5 игра,2019-04-05 19:05:00+03:00,2019-04-09 19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-04-08T23:59:59...,"{'1': 12, '2': 12, '3': 12}",2019,1.0,3,0.011535


### Генерация данных

Так как мы не разлтичаем по признакам вопросы с одного турнира между собой, то будем предсказывать долю правильных ответов на вопросы для турнира

In [8]:
def generate_data(df, do_update, total_cups=None, total_questions=None):
    X, y = [], []
    if do_update:
        total_cups, total_questions = 0, 0
    for cup_id in df.sort_values('dateStart').id:
        cup_features = get_cup_features(cup_id)
        if do_update:
            total_cups += 1
            total_questions += results[cup_id]['questions_number']
        for player_id, player_data in results[cup_id]['players_results'].items():
            player_features = get_player_features(player_id, total_cups, total_questions)
            label = player_data['answer'].count('1') / len(player_data['answer'])
            X.append(np.append(cup_features, player_features))
            y.append(label)
            if do_update:
                update_features(player_id, cup_id, player_data['position'], player_data['answer'])
    return np.array(X), np.array(y), total_cups, total_questions

In [9]:
%%time
reset_player_features()
X_train, y_train, total_cups, total_questions = generate_data(cups[cups.year == 2019], do_update=True)
X_test, y_test, _, _ = generate_data(cups[cups.year == 2020], do_update=False, 
                                     total_cups=total_cups, total_questions=total_questions)
print(f'Train size={X_train.shape}, test size={X_test.shape}')
print(f'Max players features, train: {X_train.max(axis=0)[-9:]}, test: {X_test.max(axis=0)[-9:]}')

Train size=(451783, 16), test size=(112841, 16)
Max players features, train: [0.6875     5.420535   0.67804878 9.12249228 0.97222222 1.
 0.48886945 1.         1.        ], test: [0.3362963  5.42495002 0.27552213 9.12641514 0.97222222 1.
 0.4510105  1.         1.        ]
CPU times: user 12.5 s, sys: 256 ms, total: 12.8 s
Wall time: 12.8 s


In [10]:
%%time
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_predict = linear_model.predict(X_test)
print(f'MSE={mean_squared_error(y_test, y_predict)}')

MSE=0.023110748853714367
CPU times: user 896 ms, sys: 172 ms, total: 1.07 s
Wall time: 958 ms


### Предсказание ранга команд
Так как для каждого игрока мы умеем предсказывать вероятность ответа на вопрос на турнире (условно "турнирный рейтинг игрока"), то для предсказания ранга команд можно поступить несколькими способами:
* усреднить по команде "рейтинг" игроков
* взять максимальный "рейтинг" игрока
* для каждого вопроса посчитать вероятность ответа на него как $1 - \prod_i(1-p_i)$, где $p_i$ - вероятность ответить на вопрос i-го игрока. 

Воспользуемся третьим способом. Турнирный счет команды оценим как мат ожидания числа правильных ответов:
$$R(team, tournament) = \sum_{q=1}^{Q}{1 - \prod_i(1-p_i)}=Q*(1 - \prod_i(1-p_i))$$

Так как множитель Q одинаковый для всех и не зависит от команд, то его можно опустить. Далее, так как мы хотим получить корреляцию между рангом и позицией, то лучше - меньший ранг -R. Финально:
$$R(team, tournament) = - (1 - \prod_i(1-p_i)) \simeq \prod_i(1-p_i)$$

In [11]:
from scipy.stats import spearmanr, kendalltau

def calculate_corr(df, predict_function):
    global results, total_cups, total_questions
    spearman_corr = []
    kendall_corr = []
    for cup_id in df.sort_values('dateStart').id:
        if len(results[cup_id]['teams_results']) < 2:
            continue
        cup_features = get_cup_features(cup_id)
        teams = defaultdict(list)
        for player_id, player_data in results[cup_id]['players_results'].items():
            player_features = get_player_features(player_id, total_cups, total_questions)
            features = np.append(cup_features, player_features)
            p = predict_function(features)
            teams[player_data['team']].append(p)

        predicted = []
        actual = []
        for team_id, probs in teams.items():
            rank = np.prod(1-np.array(probs))
            predicted.append(rank)
            actual.append(results[cup_id]['teams_results'][team_id])
#         print(predicted, actual, results[cup_id])
        spearman_corr.append(spearmanr(predicted, actual)[0])
        kendall_corr.append(kendalltau(predicted, actual)[0])
    return np.array(spearman_corr), np.array(kendall_corr)

In [12]:
%%time 
predict_lm = lambda features: linear_model.predict(features.reshape(1, features.size))
spearman_corr, kendall_corr = calculate_corr(cups[cups.year == 2020], predict_lm)
print(f'LinearModel results: spearman_corr={spearman_corr.mean()}, kendall_corr={kendall_corr.mean()}')

LinearModel results: spearman_corr=0.6917857045297343, kendall_corr=0.5344105705631873
CPU times: user 9.45 s, sys: 167 ms, total: 9.62 s
Wall time: 8.74 s


# EM-алгоритм

Теперь постараемся учесть командный характер соревнования: участники выступают в команде, и мы знаем только ответила ли команда на вопрос или нет. Будем считать, что если хотя бы один из участников дал правильный ответ, то и команда дала правильный ответ (здесь мы предположили, что из всех мнений почему-то команда выбирает правильный с гарантией). Это нам дает такой результат: если команда дала неверный ответ, то значит ни один из участников не дал правильного ответа.

Полная вероятность при ответе на i-ый вопрос командой из K участников:
$$p = p_1^ip_2^i...p_K^i + (1-p_1^i)p_2^i...p_K^i + (1-p_1^i)(1-p_2^i)...(1-p_K^i)$$
Где за $p_k^i$ обозначена вероятность ответить участника k на i-ый вопрос. 

Если бы у нас были скрытые переменные $z_k^i$ - знал ли участник k ответ на i-ый вопрос, то было бы легко записать правдоподобие:
$$LH(x_i, z_i) = \prod_k (p_k(x_i))^{z_i^k} * (1 - p_k(x_i))^{1-z_i^k}$$

И для всех данных (всех вопросов i)
$$LH(x, z) = \prod_i \prod_k (p_k(x_i))^{z_i^k} * (1 - p_k(x_i))^{1-z_i^k}$$
$$log LH(x, z) = \sum_i \sum_k z_i^k p_k(x_i) + (1-z_i^k)(1 - p_k(x_i))$$

Заметим теперь, что в нашей модели $p_k(x_i)$ не зависит от конкретного вопроса, а только от соревнования, в котором появился данный вопрос. Следовательно, логарифм правдоподобия можно переписать:
$$log LH(x, z) = \sum_c^{cup} \sum_q^{questions} \sum_m^{member} z_q^m p_m(x_c) + (1-z_q^m)(1 - p_m(x_c))$$

Будем пытаться предсказать вероятность $p_m(x_c)$ с помощью сигмоиды:  $p_m(x_c) = \sigma(w, \eta(x_c, m))$

Для оценки $z_q^m$ будем использовать знание о том, был ли ответ команды ($y_q$) засчитан. Если ответа команды не было, то будем считать $z_q^m=0$ исходя из предположений модели. Если же был, то оценим $z_q^m$ как вероятность дать правильный ответ игроком:
$$z_q^m = \begin{cases} \sigma(w, \eta(x_c, m)), & \mbox{if } y_q\mbox{ = 1} \\ 0, & \mbox{if } y_q\mbox{ = 0} \end{cases}$$

Подставив предложенный выше $z_q^m$ в логарифм правдоподобия, и учтя, что $p_m(x_c)$ не меняется в рамках соревнования, получим:
$$log LH(x, z) = \sum_c^{cup} n_{y_q=1} (\sum_m^{member} z_q^m p_m(x_c) + (1-z_q^m)(1 - p_m(x_c))) + n_{y_q=0}(\sum_m^{member}1 - p_m(x_c)) = $$
$$=\sum_c^{cup}\sum_m^{member} n_{y_q=1}[p_m(x_c)(2z_q^m-1) + 1 - z_q^m] + n_{y_q=0}[1 - p_m(x_c)]$$
$$=\sum_c^{cup}\sum_m^{member} p_m(x_c)(n_{y_q=1}(2z_q^m-1)-n_{y_q=0})+n_{y_q=1}(1-z_q^m) + n_{y_q=0}$$

Таким образом получили EM-алгоритм:
* Expectation:
$$z_q^m = \begin{cases} \sigma(\eta(x_c, m)), & \mbox{if } y_q\mbox{ = 1} \\ 0, & \mbox{if } y_q\mbox{ = 0} \end{cases}$$

* Maximization:
$$log LH(x, z) = \sum_c^{cup}\sum_m^{member} \sigma(w, \eta(x_c, m))(n_{y_q=1}(2z_q^m-1)-n_{y_q=0})+n_{y_q=1}(1-z_q^m) + n_{y_q=0} \rightarrow max_w$$


### Генерация данных

In [13]:
def generate_data_em(df, do_update, total_cups=None, total_questions=None):
    X, n0_array, n1_array = [], [], []
    if do_update:
        total_cups, total_questions = 0, 0
    for cup_id in df.sort_values('dateStart').id:
        cup_features = get_cup_features(cup_id)
        if do_update:
            total_cups += 1
            total_questions += results[cup_id]['questions_number']
        for player_id, player_data in results[cup_id]['players_results'].items():
            player_features = get_player_features(player_id, total_cups, total_questions)
            X.append(np.append(cup_features, player_features))
            n0_array.append(player_data['answer'].count('0'))
            n1_array.append(player_data['answer'].count('1'))
            if do_update:
                update_features(player_id, cup_id, player_data['position'], player_data['answer'])
    return np.array(X), np.array(n0_array), np.array(n1_array), total_cups, total_questions

In [14]:
%%time
reset_player_features()
X_train, n0_train, n1_train, total_cups, total_questions = generate_data_em(
    cups[cups.year == 2019], do_update=True
)

print(f'Train size={X_train.shape}, n0.shape={n0_train.shape}, n1.shape={n1_train.shape}, ' +
      'total_cups={total_cups}, total_questions={total_questions}')
print(f'Max players features, train: {X_train.max(axis=0)[-9:]}, test: {X_test.max(axis=0)[-9:]}')

Train size=(451783, 16), n0.shape=(451783,), n1.shape=(451783,), total_cups={total_cups}, total_questions={total_questions}
Max players features, train: [0.6875     5.420535   0.67804878 9.12249228 0.97222222 1.
 0.48886945 1.         1.        ], test: [0.3362963  5.42495002 0.27552213 9.12641514 0.97222222 1.
 0.4510105  1.         1.        ]
CPU times: user 10.7 s, sys: 247 ms, total: 10.9 s
Wall time: 10.9 s


In [15]:
### Реализация EM-шагов

In [19]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def e_step(w, X_features):
    return 1 / (1 + np.exp(- X_features @ w))

def get_lr(f, w, dw, max_iter=100):
    f_0 = f(w)
    alpha = 1.0
    for i in range(max_iter):
        if f(w + alpha * dw) > f_0:
            break
        alpha /= 2
    return alpha

def m_step(w, X_features, z, n1, n0, verbose=False):
    sigma = z  # because we calculate z for y=1 and p(x_cup, m) identically
    coef = n1*(2*z-1) - n0
    dw = sigma*(1-sigma)*coef @ X_features / X_features.shape[0]
    def f(w_next):
        coef = n1 * (2 * z - 1) - n0
        bias = n1 * (1 - z) + n0
        p = 1 / (1 + np.exp(- X_features @ w_next))
        return np.sum(p * coef + bias)
        
    lr = get_lr(f, w, dw)
    if verbose:
        print (f'LR={lr}, max_p={z.max()}, min_p={z.min()}, std_p={z.std()}')
    return w + lr * dw


def m_step2(w, X_features, z, n1, n0, verbose=False):
    sigma = z  # because we calculate z for y=1 and p(x_cup, m) identically
    coef = n1*(2*z-1) - n0
    dw = sigma*(1-sigma)*coef @ X_features / X_features.shape[0]
    def f(w_next):
        coef = n1 * (2 * z - 1) - n0
        bias = n1 * (1 - z) + n0
        p = 1 / (1 + np.exp(- X_features @ w_next))
        return np.sum(p * coef + bias)
        
    lr = get_lr(f, w, dw)
    if verbose:
        print (f'LR={lr}, max_p={z.max()}, min_p={z.min()}, std_p={z.std()}')
    return w + lr * dw

In [24]:
def e_step(X_features, y_command, w):
    bias = 0.0
    prediction = sigmoid(X_features @ w - bias)
    return np.minimum(prediction, y_command)


def m_step(X_features, y, weights):
    model = LinearRegression()
    model.fit(X_train, y_train, sample_weight=weights)
    return model.coef_

array([0., 1., 1.])

In [17]:
w = np.random.normal(size=X_train.shape[1])
for i in range(300):
    verbose = i % 10 == 0
    z = e_step(w, X_train)
    w_next = m_step(w, X_train, z, n1_train, n0_train, verbose)
    if verbose:
        print(f'Step {i+1}, |w_next - w|={np.linalg.norm(w_next-w)}, |w|={np.linalg.norm(w)}')
    w = w_next
print(w)

LR=1.0, max_p=0.9999997547593755, min_p=0.10502759848159389, std_p=0.1670713628323673
Step 1, |w_next - w|=2.0483533314292255, |w|=5.355822232514175
LR=1.0, max_p=0.026482775188249253, min_p=4.786686380755527e-44, std_p=0.004146757803012445
Step 11, |w_next - w|=0.060116382593712867, |w|=11.443960721735266
LR=1.0, max_p=0.014949482865997845, min_p=2.976539983977859e-44, std_p=0.0023550025734114127
Step 21, |w_next - w|=0.03524577713602998, |w|=11.609962271667731
LR=1.0, max_p=0.010345125252519214, min_p=2.163695283475776e-44, std_p=0.0016377513202543677
Step 31, |w_next - w|=0.024941115368047048, |w|=11.725910737282762
LR=1.0, max_p=0.0078833282587049, min_p=1.7070097707925508e-44, std_p=0.0012531348601385456
Step 41, |w_next - w|=0.019306185764056204, |w|=11.816077525842116
LR=1.0, max_p=0.006355575964425326, min_p=1.4130191315969637e-44, std_p=0.0010138040854009145
Step 51, |w_next - w|=0.015752573955404796, |w|=11.89030192617548
LR=1.0, max_p=0.005316977725787418, min_p=1.2073217868

### Оценка алгоритма
Воспользуемся предыдущим способом построения ранга команд на основе вероятностей ответов игроков. Только теперь предскажем вероятность с помощью полученной модели в EM алгоритме

In [18]:
%%time 

def calculate_corr(df, predict_function):
    global results, total_cups, total_questions
    spearman_corr = []
    kendall_corr = []
    for cup_id in df.sort_values('dateStart').id[:10]:
        if len(results[cup_id]['teams_results']) < 2:
            continue
        cup_features = get_cup_features(cup_id)
        teams = defaultdict(list)
        for player_id, player_data in results[cup_id]['players_results'].items():
            player_features = get_player_features(player_id, total_cups, total_questions)
            features = np.append(cup_features, player_features)
            p = predict_function(features)
            if p > 0.01:
                print(p)
            teams[player_data['team']].append(p)

        predicted = []
        actual = []
        for team_id, probs in teams.items():
            rank = np.prod(1-np.array(probs))
            predicted.append(rank)
            actual.append(results[cup_id]['teams_results'][team_id])
#         print(predicted, actual, results[cup_id])
        spearman_corr.append(spearmanr(predicted, actual)[0])
        kendall_corr.append(kendalltau(predicted, actual)[0])
    return np.array(spearman_corr), np.array(kendall_corr)

predict_em = lambda features: sigmoid(w, features)
spearman_corr, kendall_corr = calculate_corr(cups[cups.year == 2020], predict_em)
print(f'EM results: spearman_corr={spearman_corr.mean()}, kendall_corr={kendall_corr.mean()}')

EM results: spearman_corr=nan, kendall_corr=nan
CPU times: user 1 s, sys: 65.9 ms, total: 1.07 s
Wall time: 226 ms


/Users/wunder9l/opt/miniconda3/envs/made-ml/lib/python3.8/site-packages/numpy/lib/function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/wunder9l/opt/miniconda3/envs/made-ml/lib/python3.8/site-packages/numpy/lib/function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
